This is the start of a compilation of techniques to tune spark code.

General Helpful Resources:

Tips and Tricks: https://gist.github.com/dusenberrymw/30cebf98263fae206ea0ffd2cb155813
    
Cloudera Tuning Part 1:
https://blog.cloudera.com/blog/2015/03/how-to-tune-your-apache-spark-jobs-part-1/
   
Cloudera Tuning Part 2:
https://blog.cloudera.com/blog/2015/03/how-to-tune-your-apache-spark-jobs-part-2/

Spark Documentation: https://spark.apache.org/docs/latest/tuning.html

Really Good Series of Courses: https://www.coursera.org/specializations/scala

This is super good: https://www.gitbook.com/book/jaceklaskowski/mastering-apache-spark/details


Data Serialization

Spark's default serializer is the Java serializer which has been known to be innefficient, consuming large percentage of CPU and RAM. The Kyro Serializer is the better option as it allows flexibility to register custom classes.

In [7]:
#Changing the serializer to kyro in the spark conf
val conf = new SparkConf()
      .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

#Controlling size of kyro buffer
val conf = new SparkConf()
      .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
      .set("spark.kryoserializer.buffer.mb","24") 

SyntaxError: invalid syntax (<ipython-input-7-2f2b0abaa206>, line 2)

Garbage Collection

Use Data Structures with fewer objects

Persist object in serialized form

Clean up cached collections after they are no longer being used

https://databricks.com/blog/2015/05/28/tuning-java-garbage-collection-for-spark-applications.html

In [8]:
#Measuring Java GC
-verbose:gc - XX:+PrintGCDetails
XX:+PrintGCTimeStamps

SyntaxError: invalid syntax (<ipython-input-8-6b7845ba6d61>, line 2)

Memory Management

Use spark.memory.fraction to set percentage of heap space used

Don't use executors with too much memory (it delays garbace collection)

Don't use lots of small executors

In [9]:
#set pointers to 4 rather than 8 bytes
-XX:+UseCompressedOops

SyntaxError: invalid syntax (<ipython-input-9-ef47ccedbacd>, line 2)

Parallelism

Recomended 2-3 tasks per core

** Optimize the number of partitions (General Rule 128Mb per partition) **
Coursera Course Goes into Great Detail!!!

Types of partitioning: https://acadgild.com/blog/partitioning-in-spark/

Hash Partitioning

Range Partitioning

*Partitioning improves operations that shuffle by key*

repartition() is expensive as it moves data around, use coalesce() instead to decrease the number of partitions

Transformations

Maximize usage of transfomrations such as map() and filter()

Minimize usage of wide transformations like: groupByKey(), reduceByKey(), join()

Use cogroup() to join grouped datasets

Use repartitionAndSortWithinPartitions()

https://stackoverflow.com/questions/37227286/how-to-use-sparks-repartitionandsortwithinpartitions


Caching

Cache collections that are to be used multiple times

Cache serialized objects intstead of raw data, as it improves garbace collection time


Excellent Resources:


http://sujee.net/2015/01/22/understanding-spark-caching/#.WfYNNsaZMy4


https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-rdd-caching.html

When faced with an *OutofMemoryError* try to increase parallelism to more than the number of cores